In [0]:
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

# model
from sklearn.ensemble import RandomForestRegressor

SEED=42

In [0]:
path = 'drive/My Drive/bigcontest2019/data/'

train_label = pd.read_csv(path + 'train/train_label.csv')
train_activity = pd.read_csv(path + 'train/train_activity.csv')
train_combat = pd.read_csv(path + 'train/train_combat.csv')
train_payment = pd.read_csv(path + 'train/train_payment.csv')
train_pledge = pd.read_csv(path + 'train/train_pledge.csv')
train_trade = pd.read_csv(path + 'train/train_trade.csv')

test1_activity = pd.read_csv(path + 'test/test1_activity.csv')
test1_combat = pd.read_csv(path + 'test/test1_combat.csv')
test1_payment = pd.read_csv(path + 'test/test1_payment.csv')
test1_pledge = pd.read_csv(path + 'test/test1_pledge.csv')
test1_trade = pd.read_csv(path + 'test/test1_trade.csv')

test2_activity = pd.read_csv(path + 'test/test2_activity.csv')
test2_combat = pd.read_csv(path + 'test/test2_combat.csv')
test2_payment = pd.read_csv(path + 'test/test2_payment.csv')
test2_pledge = pd.read_csv(path + 'test/test2_pledge.csv')
test2_trade = pd.read_csv(path + 'test/test2_trade.csv')

In [5]:
train_activity_merge = train_activity.groupby('acc_id').agg({'day':'nunique', 
                                                             'char_id':'nunique',
                                                             'server':'nunique', 
                                                             'playtime':'sum', 
                                                             'npc_kill':'sum', 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'}).reset_index(drop=True)

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,1,75001,397380,aa,1.441844,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,1.10514,0.034006,0.0
1,1,75001,216231,aa,0.283219,2.247978,0.047085,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,-0.013963,0.0
2,1,75711,308769,aa,1.036910,2.956638,0.321821,0.166593,0.003470,1,0.245883,0.247337,0.0,0.000000,0.00000,-0.001987,0.0
3,1,72230,387177,aa,0.229384,4.042102,0.099147,0.000000,0.001735,0,0.000000,0.000000,0.0,0.000000,0.00000,-0.028884,0.0
4,1,34253,339862,aa,1.088405,0.596802,0.003377,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,0.011777,0.0
5,1,83200,109264,aa,0.074901,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0
6,1,83200,396160,aa,0.245769,0.000000,0.047436,0.000000,0.000000,0,0.000000,0.000000,0.0,0.241790,0.00000,0.000000,0.0
7,1,13896,302284,aa,0.004681,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0
8,1,13896,95032,aa,0.414296,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,-0.000337,0.0
9,1,13896,374971,aa,0.007022,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.00000,-0.141990,0.0
